# Demo de dos modelos de lengua

<a target="_blank" href="https://colab.research.google.com/github/jaspock/me/blob/main/docs/materials/assets/misterios/notebooks/llama.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> <a href="http://dlsi.ua.es/~japerez/"><img src="https://img.shields.io/badge/Universitat-d'Alacant-5b7c99" style="margin-left:10px"></a>

Cuaderno escrito por Juan Antonio Pérez en 2025. Este cuaderno permite evaluar las continuaciones y las probabilidades de la siguiente palabra según dos modelos de lengua, uno entrenado únicamente para predecir el siguiente token y otro entrenado para seguir instrucciones.

El entorno de ejecución de este cuaderno ha de tener una GPU. En Google Colab, la puedes conseguir desde el menú *Entorno de ejecución* / *Cambiar tipo de entorno de ejecución*.

## Inicialización de los modelos

El siguiente código inicializa el modelo base y el modelo que sigue instrucciones a partir de dos modelos abiertos de tamaño 1.5B (1,5 millardos de parámetros). La descarga de los modelos puede llevar unos minutos, especialmente la primera vez que se ejecute el cuaderno en un nuevo entorno de ejecución.

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Base model setup
# base_model_id = "Qwen/Qwen2.5-0.5B"
base_model_id = "Qwen/Qwen2.5-1.5B"
base_pipe = pipeline(
    "text-generation",
    model=base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                  torch_dtype=torch.bfloat16, device_map="auto")

# Instruction-tuned model setup
# instruction_model_id = "Qwen/Qwen2.5-0.5B-Instruct"
instruction_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
instruction_pipe = pipeline(
    "text-generation",
    model=instruction_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
instruction_tokenizer = AutoTokenizer.from_pretrained(instruction_model_id)
instruction_model = AutoModelForCausalLM.from_pretrained(instruction_model_id,
                                                         torch_dtype=torch.bfloat16, device_map="auto")

## Obtención de las probabilidades de salida

La función `get_word_probs` devuelve un diccionario de Python con las probabilidades del modelo para las palabras de la lista `candidates` cuando se ha procesado el contexto `text` con el modelo `model`.

In [ ]:
def get_word_probs(model, tokenizer, text, candidates):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    logits = outputs.logits[0, -1]  # last token logits
    probs = torch.softmax(logits, dim=-1)

    candidate_probs = {}  # initial dictionary
    for word in candidates:
        token_id = tokenizer.convert_tokens_to_ids(word)
        if token_id is not None:
            candidate_probs[word] = probs[token_id].item()
        else:
            candidate_probs[word] = 0.0  # word not in embedding table

    return candidate_probs

## Probando, probando...

En primer lugar, probaremos que los modelos funcionan pidiéndoles que continúen un texto.

Si el valor de `do_sample` está a `True`, el sistema genera salidas diferentes cada vez. La diversidad en ese caso se puede ajustar jugando con los argumentos siguientes:

- `top_k`: limita el número de palabras (tokens) más probables a considerar; por ejemplo, `top_k=50`
- `top_p`: elige palabras hasta alcanzar una probabilidad acumulada específica; por ejemplo, `top_p=0.95`
- `temperature`: controla la aleatoriedad: valores bajos entre 0 y 1 son más conservadores, altos entre 1 y 2 más creativos, pero aumentando el riesgo de incoherencias; es habitual hacer `temperature=0.7`.

In [ ]:
question = "What is the capital of France?"

inputs_normal = base_tokenizer(question, return_tensors="pt").to("cuda")
response_normal = base_model.generate(**inputs_normal, max_new_tokens=50)
print("🡪 Response of the base model:" , base_tokenizer.decode(response_normal[0], skip_special_tokens=True, do_sample=True))

inputs_instr = instruction_tokenizer(question, return_tensors="pt").to("cuda")
response_instr = instruction_model.generate(**inputs_instr, max_new_tokens=50)
print("🡪 Response of the instruction-tuned model: ", instruction_tokenizer.decode(response_instr[0], skip_special_tokens=True, do_sample=True))

## Obtención de probabilidades

A continuación, usaremos los modelos para obtener las probabilidades de la siguiente palabra una vez procesado el texto de `example-text`, pero solo para aquellas palabras de la lista `candidate_words`.

En realidad, los modelos de lengua no trabajan necesariamente a nivel de palabra, sino que por ciertos motivos que no vienen al caso, trocean las palabras pocos frecuentes en fragmentos que sí son frecuentes. A los elementos del vocabulario resultantes (que pueden ser palabras completas o fragmentos de ellas) se les denomina *tokens*.

Por ello, asegúrate de que `candidate_words` contiene palabras que sí están en el vocabulario; una manera rápida de comprobarlo es verificar que no se devuelve 0.0 para su probabilidad, ya que es lo que hace el programa si la palabra no está en el vocabulario. Aunque en un principio podría parecer que esto no nos permite diferenciar entre palabras que no están en el vocabulario y palabras que sí lo están pero tienen probabilidad nula, lo cierto es que, por las funciones matemáticas que usa la red neuronal, la salida de las palabras poco probables será un valor muy pequeño, pero nunca cero.

Observa que las probabilidades se presentan en notación científica, por ejemplo, `7.59027898311615e-08`. Esto equivale a $7.59027898311615 \times 10^{-8}$ o, lo que es lo mismo, 0.0000000759027898311615.

Aunque el código obtiene el valor de las probabilidades de que las palabras de `candidate_words` sean la palabra que sigue a `example_text`, lo cierto es que este enfoque no es completamente válido para el modelo que sigue instrucciones, ya que este espera que la entrada siga una determinada plantilla con las instrucciones generales dadas al modelo (`system`) y la entrada del usuario (`user`). Por ello, además de simplemente obtener las probabilidades emitidas por la red neuronal (algo que puede dar resultados más o menos correctos, ya que el modelo fue entrenado en su fase inicial para predecir el siguiente token), usamos esta plantilla para pedir al modelo una continuación siguiendo las mismas pautas con las que ha sido entrenado.

In [ ]:
example_text = "The weather in this summer day is "
candidate_words = ["hot", "cold", "nice", "car"]

probs_base = get_word_probs(base_model, base_tokenizer, example_text, candidate_words)
print("🡪 Probabilities of the base model:", probs_base)

instruction_text = example_text
probs_instr = get_word_probs(instruction_model, instruction_tokenizer, instruction_text, candidate_words)
print("🡪 Probabilities of the instruction-tuned model with no template:", probs_instr)

system = "You are an expert who knows how to complete every incomplete sentence."
prompt1 = "Hi! I will ask you later about the following sentence: '" + example_text + "'. For now, just say that you agree."
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt1},
]
text = instruction_tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = instruction_tokenizer([text], return_tensors="pt").to(instruction_model.device)

generated_ids = instruction_model.generate(**inputs, max_new_tokens=40)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
]

response1 = instruction_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("🡪 First output of the instruction-tuned model following the template:", response1)

prompt2= "Gime me four comma-separated words that would likely continue the sentence I gave you before."
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt1},
    {"role": "assistant", "content": response1},
    {"role": "user", "content": prompt2},
]
text = instruction_tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = instruction_tokenizer([text], return_tensors="pt").to(instruction_model.device)

generated_ids = instruction_model.generate(**inputs, max_new_tokens=40)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
]

response2 = instruction_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("🡪 Second output of the instruction-tuned model following the template:", response2)

## Generación de respuestas

Por último, generaremos más salidas. En el caso del modelo que sigue instrucciones, usaremos la plantilla que se usó durante su entrenamiento para obtener mejores resultados.

Para las personas muy observadoras, en el código anterior hemos usado por separado el *tokenizador* (que obtiene los *tokens* asociados a las palabras) y el modelo en sí. En el código siguiente, usaremos un *pipeline* que simplifica el código para obtener la salida del modelo, pero no nos permite obtener de forma directa las probabilidades de salida que nos interesaban antes.

In [ ]:
output = base_pipe("The key to life is", max_new_tokens=40, do_sample=True)
print("🡪 Response of the base model: ", output[0]["generated_text"])

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
outputs = instruction_pipe(messages, max_new_tokens=40, do_sample=True)
print("🡪 Response of the instruction-tuned model: ", outputs[0]["generated_text"][-1])   # print only new output

# call apply_chat_template if not using the pipeline:
# prompt = base_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)